## Reading the profiles in UI

In [ ]:
import duckdb
import pandas as pd
df = duckdb.sql("select * from read_parquet('person_profiles.parquet/*.parquet') where PersonID = 53380").fetchdf()
import json
deets = json.loads(df['Details'].values[0])

In [54]:
import pyspark
from pyspark.sql.functions import udf, col, size, collect_set, collect_list
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from collections import Counter
from pyspark.sql.functions import col, collect_set, struct,count
from pyspark.sql.types import StructType, StructField, IntegerType
from datetime import date
import statistics
import json
import calendar

import polars as pl
spark = SparkSession.builder.appName('customer360.ai').getOrCreate()


df = spark.read.parquet("cdp_profiles.parquet")


In [55]:

def calculate_age(born_str):
    born = date.fromisoformat(born_str)
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))
@udf 
def mode_udf(column_arr):
    return statistics.mode(column_arr)


## tried something different

Timelines and anything to do with dates have to be properly sorted.
> currently has some bugs

In [56]:
@udf()
def sum_ages(first_name,last_name, email,mobile, revenue,booker_type,city,total_passengers,bookerid,
 booking_date,travelseat,meals,destination,channel,isemployee,baggage,gender,employee_related,insurance,
 origin,travelby,dob,isregistered, bookingcurrency, passenger_hash):
    # build a polars dataframe from the column arrays
    df = pl.DataFrame({
        'first_name': first_name,
        'last_name': last_name,
        'email': email,
        'mobile': mobile,
        'revenue': revenue,
        'booker_type': booker_type,
        'city': city,
        'total_passengers': total_passengers,
        'bookerid': bookerid,
        'booking_date': booking_date,
        'travelseat': travelseat,
        'meals': meals,
        'destination': destination,
        'channel': channel,
        'isemployee': isemployee,
        'baggage': baggage,
        'gender': gender,
        'employee_related': employee_related,
        'insurance': insurance,
        'origin': origin,
        'travelby': travelby,
        'dob': dob,
        'isregistered': isregistered,
        'booking_currency': bookingcurrency,
        'passenger_hash': passenger_hash
    })
    # sort on booking date
    df = df.sort('booking_date')
    first_date = df['booking_date'].min()
    last_date = df['booking_date'].max()
    exchange_rates= {'THB': 0.657845, 'AED': 13.203984, 'MYR': 7.624044, 'EUR': 0.011741, 'OMR': 0.001127, 'KWD': 0.003423, 'VND': 254.181015, 'USD': 0.019467, 'AUD': 0.025128, 'CNY': 0.126146, 'PHP': 1.0, 'KRW': 22.734294, 'JPY': 2.180405, 'HKD': 0.151516, 'TWD': 0.539236, 'QAR': 0.071024, 'MOP': 0.156013, 'IDR': 278.384131, 'SAR': 0.073001, 'INR': 1.427856, 'BHD': 0.007356, 'SGD': 0.026176, 'BND': 0.026101}
    def transform_revenue(booking_currency, revenue):
        # booking_currency = row.get('bookingcurrency')
        rev = []
        for idx, each_booking_currency in enumerate(booking_currency): 
            if each_booking_currency in exchange_rates:
                exchange_rate = exchange_rates[each_booking_currency]
                transformed_revenue = revenue[idx] * exchange_rate
                rev.append(transformed_revenue)
            else:
                rev.append(revenue[idx])
        return pl.Series('revenue', rev) 
    df = df.with_columns([
        transform_revenue(df['booking_currency'], df['revenue'])
    ])
    # df['revenue'] = df.apply(transform_revenue)
    # iterate over booking_date
    booking_freq = {}
    # for date in df['booking_date'].to_list():
    #     # date is %d-%m-%Y
    #     month = date.split('-')[1]
    #     try:
    #         month_name = calendar.month_name[int(month)]
    #     except:
    #         month_name = "Unknown"
    #     if month in booking_freq:            
    #         booking_freq[month_name] += 1
    #     else:
    #         booking_freq[month] = 1
    booking_freq = {
        'jan': 0,
        'feb': 0,
        'mar': 0,
        'apr': 0,
        'may': 0,
        'jun': 0,
        'jul': 0,
        'aug': 0,
        'sep': 0,
        'oct': 0,
        'nov': 0,
        'dec': 0,
    }
    dates = df['booking_date'].to_list()
    for date in dates:
        if date.split('-')[1] == '01':
            booking_freq['jan'] += 1
        elif date.split('-')[1] == '02':
            booking_freq['feb'] += 1
        elif date.split('-')[1] == '03':
            booking_freq['mar'] += 1
        elif date.split('-')[1] == '04':
            booking_freq['apr'] += 1
        elif date.split('-')[1] == '05':
            booking_freq['may'] += 1
        elif date.split('-')[1] == '06':
            booking_freq['jun'] += 1
        elif date.split('-')[1] == '07':
            booking_freq['jul'] += 1
        elif date.split('-')[1] == '08':
            booking_freq['aug'] += 1
        elif date.split('-')[1] == '09':
            booking_freq['sep'] += 1
        elif date.split('-')[1] == '10':
            booking_freq['oct'] += 1
        elif date.split('-')[1] == '11':
            booking_freq['nov'] += 1
        elif date.split('-')[1] == '12':
            booking_freq['dec'] += 1
    timeline = []
    # loop on last 7 records in reverse order
    for i in range(1,8):
        row = df.tail(1)
        timeline.append({
            'origin': row['origin'],
            'destination': row['destination'],
            'channel': row['channel'],
            'baggage': row['baggage'],
            'meals': row['meals'],
            'booking_date': row['booking_date'],
            'revenue': row['booking_currency'] + row['revenue'],
        }
    )
    dobss = []
    for db in dob:
        if db != 'Unknown':
            dobss.append(calculate_age((db)))
        else:
            dobss.append('Unknown')
    
    age_groups = range(1, 111, 10)

    age_group_counts = {f'{i}-{i+9}': 0 for i in age_groups}
    age_group_counts['unknown'] = 0
    for age in dobss:
        if age == 'Unknown':
            age_group_counts['unknown'] += 1
        for i in age_groups:
            if age in range(i, i+10):
                age_group_counts[f'{i}-{i+9}'] += 1
    # get most frequent first name
    def mode(column_arr):
        return statistics.mode(column_arr)
    temp = {
        "first_name" : mode(first_name),
        "last_name" : mode(last_name),
        "email" : mode(email),
        "mobile" : mode(mobile),
        "booker_type" : mode(booker_type),
        "city" : mode(city),
        "total_passengers" : df.shape[0],
        "unique" : df['passenger_hash'].unique().shape[0],
        "first_date" : first_date,
        "last_date" : last_date,
        "booking_freq" : booking_freq,
        "seat_counts" : dict(Counter(travelseat)),
        "meals_count" : dict(Counter(meals)),
        "destination_count" : dict(Counter(destination)),
        "channel_count" : dict(Counter(channel)),
        "employee_count" : dict(Counter(isemployee)),
        "baggage_count" : dict(Counter(baggage)),
        "gender_count" : dict(Counter(gender)),
        "employee_related_count" : dict(Counter(employee_related)),
        "insurance_count" : dict(Counter(insurance)),
        "origin_count" : dict(Counter(origin)),
        "travelby_count" : dict(Counter(travelby)),
        "isregistered_count" : dict(Counter(isregistered)),
        "revenue" : df['revenue'].sum(),
        "timeline" : timeline,
    }
    print(temp)
    return json.dumps(temp)

# pyspark fill null values with Unknown
df = df.fillna("Unknown")

result = df.groupBy(("PersonID")).agg(
    mode_udf(F.collect_list("BookerFirstName")).alias("First_Name"),mode_udf(F.collect_list("BookerLastName")).alias("Last_Name"),
mode_udf(F.collect_list("BookerMobile")).alias("Mobile"),
mode_udf(F.collect_list("BookerEmailAddress")).alias("Email")
,sum_ages(F.collect_list("BookerFirstName"),F.collect_list("BookerLastName"),F.collect_list("BookerEmailAddress"),
F.collect_list("BookerMobile"),F.collect_list("Revenue"),F.collect_list("BookerType"),F.collect_list("BookerCity"),
F.collect_list("PassengerID"),F.collect_list("PersonID"),F.collect_list("BookingDate"),F.collect_list("TravelSeat"),
F.collect_list("TravelMeals"),F.collect_list("TravelDestination"),F.collect_list("BookingChannel"),F.collect_list("IsEmployee"),
F.collect_list("TravelBaggage"),F.collect_list("Gender"),F.collect_list("IsEmployeeDependent"),F.collect_list("TravelInsurance"),
F.collect_list("TravelOrigin"),F.collect_list("TravelSoloOrGroup"), F.collect_list("DateOfBirth"),F.collect_list("IsRegistered"),F.collect_list("BookingCurrency"),F.collect_list("passenger_hash")).alias("Details"))
result.write.parquet("person_profiles.parquet", mode="overwrite")

{'first_name': 'xxxx', 'last_name': 'xxxx', 'email': 'tXXX@XXXX.com', 'mobile': 'Unknown', 'booker_type': 'Agent', 'city': 'Unknown', 'total_passengers': 1, 'unique': 1, 'first_date': '2022-10-06 12:37:57.18', 'last_date': '2022-10-06 12:37:57.18', 'booking_freq': {'jan': 0, 'feb': 0, 'mar': 0, 'apr': 0, 'may': 0, 'jun': 0, 'jul': 0, 'aug': 0, 'sep': 0, 'oct': 1, 'nov': 0, 'dec': 0}, 'seat_counts': {'21D_2F_9E_31E': 1}, 'meals_count': {'None': 1}, 'destination_count': {'BCD_MNL_MNL_SIN': 1}, 'channel_count': {'API': 1}, 'employee_count': {'No': 1}, 'baggage_count': {'Checked Baggage Allowance - 20 Kilos': 1}, 'gender_count': {'2': 1}, 'employee_related_count': {'No': 1}, 'insurance_count': {'None': 1}, 'origin_count': {'SIN': 1}, 'travelby_count': {'Solo': 1}, 'isregistered_count': {'No': 1}, 'revenue': 13.596599679999999, 'timeline': [{'origin': shape: (1,)
Series: 'origin' [str]
[
	"SIN"
], 'destination': shape: (1,)
Series: 'destination' [str]
[
	"BCD_MNL_MNL_SI…
], 'channel': shape

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_6942/4051122333.py", line 168, in sum_ages
  File "/home/gitpod/.pyenv/versions/3.11.1/lib/python3.11/json/__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/gitpod/.pyenv/versions/3.11.1/lib/python3.11/json/encoder.py", line 200, in encode
    chunks = self.iterencode(o, _one_shot=True)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/gitpod/.pyenv/versions/3.11.1/lib/python3.11/json/encoder.py", line 258, in iterencode
    return _iterencode(o, 0)
           ^^^^^^^^^^^^^^^^^
  File "/home/gitpod/.pyenv/versions/3.11.1/lib/python3.11/json/encoder.py", line 180, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type Series is not JSON serializable


## Made some modifications to previous code

In [ ]:
import pyspark
from pyspark.sql.functions import udf, col, size, collect_set, collect_list
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from collections import Counter
from pyspark.sql.functions import col, collect_set, struct,count
from pyspark.sql.types import StructType, StructField, IntegerType
from datetime import date
import statistics
import json
import polars as pl
spark = SparkSession.builder.appName('customer360.ai').getOrCreate()


df = spark.read.parquet("cdp_profiles.parquet")


def calculate_age(born_str):
    born = date.fromisoformat(born_str)
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

@udf 
def abs(column_arr):
    return statistics.mode(column_arr)

@udf()
def sum_ages(first_name,last_name, email,mobile, revenue,booker_type,city,total_passengers,bookerid,
 booking_date,travelseat,meals,destination,channel,isemployee,baggage,gender,employee_related,insurance,
 origin,travelby,dob,isregistered,bookingcurrency, passenger_hash):
    # build a polars dataframe from the column arrays
    df = pl.DataFrame({
        'first_name': first_name,
        'last_name': last_name,
        'email': email,
        'mobile': mobile,
        'revenue': revenue,
        'booker_type': booker_type,
        'city': city,
        'total_passengers': total_passengers,
        'bookerid': bookerid,
        'booking_date': booking_date,
        'travelseat': travelseat,
        'meals': meals,
        'destination': destination,
        'channel': channel,
        'isemployee': isemployee,
        'baggage': baggage,
        'gender': gender,
        'employee_related': employee_related,
        'insurance': insurance,
        'origin': origin,
        'travelby': travelby,
        'dob': dob,
        'isregistered': isregistered
    })
    # sort on booking date
    df = df.sort('booking_date')

    first_date = df['booking_date'].min()
    last_date = df['booking_date'].max()
    dicts = {
        'jan': 0,
        'feb': 0,
        'mar': 0,
        'apr': 0,
        'may': 0,
        'jun': 0,
        'jul': 0,
        'aug': 0,
        'sep': 0,
        'oct': 0,
        'nov': 0,
        'dec': 0,
    }
    dates = df['booking_date'].to_list()
    for date in dates:
        if date.split('-')[1] == '01':
            dicts['jan'] += 1
        elif date.split('-')[1] == '02':
            dicts['feb'] += 1
        elif date.split('-')[1] == '03':
            dicts['mar'] += 1
        elif date.split('-')[1] == '04':
            dicts['apr'] += 1
        elif date.split('-')[1] == '05':
            dicts['may'] += 1
        elif date.split('-')[1] == '06':
            dicts['jun'] += 1
        elif date.split('-')[1] == '07':
            dicts['jul'] += 1
        elif date.split('-')[1] == '08':
            dicts['aug'] += 1
        elif date.split('-')[1] == '09':
            dicts['sep'] += 1
        elif date.split('-')[1] == '10':
            dicts['oct'] += 1
        elif date.split('-')[1] == '11':
            dicts['nov'] += 1
        elif date.split('-')[1] == '12':
            dicts['dec'] += 1

    booking_dates = []

    if(len(dates)<6):
        booking_dates = dates
    else:
        booking_dates = dates[-6:]
    
    timeline = {}
    for i in range(len(booking_dates)):
        index = dates.index(booking_dates[i])
        timeline[i+1] = {
            'origin': origin[index],
            'destination': destination[index],
            'channel':channel[index],
            'baggage':baggage[index],
            'meals':meals[index],
            'booking_date':booking_date[index],
            'revenue':revenue[index]
        }

    dobss = []
    for db in dob:
        if db != 'Unknown':
            dobss.append(calculate_age((db)))
        else:
            dobss.append('Unknown')
    
    age_groups = range(1, 111, 10)

    age_group_counts = {f'{i}-{i+9}': 0 for i in age_groups}
    age_group_counts['unknown'] = 0
    for age in dobss:
        if age == 'Unknown':
            age_group_counts['unknown'] += 1
        for i in age_groups:
            if age in range(i, i+10):
                age_group_counts[f'{i}-{i+9}'] += 1


    seat_counts = dict(Counter(travelseat))
    meals_count = dict(Counter(meals))
    destination_count = dict(Counter(destination))
    channel_count = dict(Counter(channel))
    employee_count = dict(Counter(isemployee))
    baggage_count = dict(Counter(baggage))
    gender_count = dict(Counter(gender))
    employee_related_count = dict(Counter(employee_related))
    insurance_count = dict(Counter(insurance))
    origin_count = dict(Counter(origin))
    travelby_count = dict(Counter(travelby))
    isregistered_count = dict(Counter(isregistered))
        
    temp = {'first_name': first_name[0],'last_name' :last_name[0],'email' :email[0],'mobile' :mobile[0],'revenue': sum(revenue),
    'booker_type':booker_type[0],'city':city[0],'total_passengers':len(total_passengers),
    'bookerid':bookerid[0], 'first_date': first_date, 'last_date': last_date,'dicts':dicts,'travelseat':seat_counts,
    'meals':meals_count,'destination':destination_count,'channel':channel_count,'isemployee':employee_count,
    'baggage':baggage_count,'gender':gender_count,'employee_related':employee_related_count,'insurance':insurance_count,
    'origin':origin_count,'travelby':travelby_count, 'dob': age_group_counts,'isregistered':isregistered_count,'timeline':timeline}
    details = json.dumps(temp)
    return details
result = df.groupBy(("PersonID")).agg(
    abs(F.collect_list("BookerFirstName")).alias("First_Name"),abs(F.collect_list("BookerLastName")).alias("Last_Name"),
abs(F.collect_list("BookerMobile")).alias("Mobile"),
abs(F.collect_list("BookerEmailAddress")).alias("Email")
,sum_ages(F.collect_list("BookerFirstName"),F.collect_list("BookerLastName"),F.collect_list("BookerEmailAddress"),
F.collect_list("BookerMobile"),F.collect_list("Revenue"),F.collect_list("BookerType"),F.collect_list("BookerCity"),
F.collect_list("PassengerID"),F.collect_list("PersonID"),F.collect_list("BookingDate"),F.collect_list("TravelSeat"),
F.collect_list("TravelMeals"),F.collect_list("TravelDestination"),F.collect_list("BookingChannel"),F.collect_list("IsEmployee"),
F.collect_list("TravelBaggage"),F.collect_list("Gender"),F.collect_list("IsEmployeeDependent"),F.collect_list("TravelInsurance"),
F.collect_list("TravelOrigin"),F.collect_list("TravelSoloOrGroup"), F.collect_list("DateOfBirth"),F.collect_list("IsRegistered"),F.collect_list("BookingCurrency"),F.collect_list("passenger_hash")).alias("Details"))
# result.show(truncate=False)

# save the result to a parquet file
result.write.parquet("person_profiles.parquet", mode="overwrite")


@udf()
def passenger_queries(first_name,middle_name,last_name,
dob,gender,phone,email,city,
state,zipcode,nation,key,booking_date,revenue,
isregistered,passengerid,baggage,travelby,channel,
insurance,origin,meals,travelseat,destination):

    name = first_name[0] + " " + middle_name[0] + " "+last_name[0]
    bookings_counts = dict(Counter(passengerid))
    dates =[x.split(" ")[0] for x in booking_date]
    # sort the dates and get the first and last date
    first_date = sorted(dates)[0]
    last_date = sorted(dates)[-1]
    total_revenue = sum(revenue)
    isregistered_count = dict(Counter(isregistered))
    dicts = {
        'jan': 0,
        'feb': 0,
        'mar': 0,
        'apr': 0,
        'may': 0,
        'jun': 0,
        'jul': 0,
        'aug': 0,
        'sep': 0,
        'oct': 0,
        'nov': 0,
        'dec': 0,
    }
    for date in dates:
        if date.split('-')[1] == '01':
            dicts['jan'] += 1
        elif date.split('-')[1] == '02':
            dicts['feb'] += 1
        elif date.split('-')[1] == '03':
            dicts['mar'] += 1
        elif date.split('-')[1] == '04':
            dicts['apr'] += 1
        elif date.split('-')[1] == '05':
            dicts['may'] += 1
        elif date.split('-')[1] == '06':
            dicts['jun'] += 1
        elif date.split('-')[1] == '07':
            dicts['jul'] += 1
        elif date.split('-')[1] == '08':
            dicts['aug'] += 1
        elif date.split('-')[1] == '09':
            dicts['sep'] += 1
        elif date.split('-')[1] == '10':
            dicts['oct'] += 1
        elif date.split('-')[1] == '11':
            dicts['nov'] += 1
        elif date.split('-')[1] == '12':
            dicts['dec'] += 1

    baggage_count = dict(Counter(baggage))
    travelby_count = dict(Counter(travelby))
    channel_count = dict(Counter(channel))
    insurance_count = dict(Counter(insurance))
    origin_count = dict(Counter(origin))
    meals_count = dict(Counter(meals))
    seat_counts = dict(Counter(travelseat))
    destination_counts = dict(Counter(destination))
    booking_dates = []

    if(len(dates)<6):
        booking_dates = dates
    else:
        booking_dates = dates[-6:]
    
    timeline = {}
    for i in range(len(booking_dates)):
        index = dates.index(booking_dates[i])
        timeline[i+1] = {
            'origin': origin[index],
            'destination': destination[index],
            'channel':channel[index],
            'baggage':baggage[index],
            'meals':meals[index],
            'booking_date':booking_date[index],
            'revenue':revenue[index]
        }
    



    temp_pass = {'fullname': name,'dob':dob,'gender':gender,'phone':phone,'email':email,'city':city,'state':state,
    'zipcode':zipcode,'nation':nation,'key':key,'first_date': first_date, 'last_date': last_date,
    'revenue':total_revenue,'isregistered':isregistered_count,'passengerid':passengerid,'dicts':dicts,'baggage':baggage_count,
    'travelby':travelby_count,'channel':channel_count,'insurance':insurance_count,'origin':origin_count,'meals':meals_count,'travelseat':seat_counts,
    'destination':destination_counts,'timeline':timeline}
    return json.dumps(temp_pass)

result_passenger = df.groupBy("PassengerID").agg(passenger_queries(F.collect_list("FirstName")
,F.collect_list("MiddleName"),F.collect_list("LastName"),F.collect_list("DateOfBirth"),
F.collect_list("Gender"),F.collect_list("Phone"),F.collect_list("EmailAddress"),F.collect_list("City"),
F.collect_list("State"),F.collect_list("ZipCode"),F.collect_list("Nationality"),F.collect_list("ProvisionalPrimaryKey"),
F.collect_list("BookingDate"),F.collect_list('Revenue'),F.collect_list('IsRegistered'),F.collect_list('PassengerID'),
F.collect_list("TravelBaggage"),F.collect_list("TravelSoloOrGroup"),F.collect_list("BookingChannel"),F.collect_list("TravelInsurance"),
F.collect_list("TravelOrigin"),F.collect_list("TravelMeals"),F.collect_list("TravelSeat"),F.collect_list("TravelDestination")).alias("results"))

# result_passenger.show(truncate=False)
result_passenger.write.parquet("passenger_profiles.parquet", mode="overwrite")



(1, -1)
(2, 5)
(3, 8)


column_0,column_1
i64,i64
2,-3
4,15
6,24
